In [12]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import FAISS
from typing import Type, Dict
from pydantic import BaseModel, Field
from langchain.output_parsers import JsonOutputToolsParser
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType

from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import SimpleSequentialChain,StuffDocumentsChain, SequentialChain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

from dotenv.main import load_dotenv

from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

from langchain.chains.router import MultiPromptChain
from langchain.chains import ConversationChain
load_dotenv()
# decouple to read .env variables(OpenAI Key)
#from decouple import config
# LLM Router chains
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
import os

In [ ]:
import pandas as pd

data = [
    {"apartment_id": 1, "number": 101, "address": "123 Main Street"},
    {"apartment_id": 2, "number": 102, "address": "456 Elm Street"},
    {"apartment_id": 3, "number": 103, "address": "789 Oak Avenue"},
    {"apartment_id": 4, "number": 104, "address": "321 Pine Road"},
    {"apartment_id": 5, "number": 105, "address": "654 Maple Lane"},
    {"apartment_id": 6, "number": 106, "address": "987 Cedar Court"},
    {"apartment_id": 7, "number": 107, "address": "246 Birch Drive"},
    {"apartment_id": 8, "number": 108, "address": "135 Walnut Way"},
    {"apartment_id": 9, "number": 109, "address": "369 Cherry Lane"},
    {"apartment_id": 10, "number": 110, "address": "802 Ash Street"}
]

df = pd.DataFrame(data)
df.to_csv('apartments.csv', index=False)


In [13]:
chat_llm = ChatOpenAI(api_key=os.environ['OPENAI_API_KEY'], model='gpt-4', temperature=0.34)

## Load data

In [15]:
loader = CSVLoader(file_path='apartments.csv', encoding="utf-8", csv_args={
                'delimiter': ','})

data = loader.load()

embeddings = OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
vectorstore = FAISS.from_documents(data, embeddings)

## Knowledge retrieval agent/Chain

In [16]:
from langchain.tools.retriever import create_retriever_tool

retriever = vectorstore.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    "apartment info",
    "Search for information about an apartment. For any questions about apartment information, you must use this tool!",
)


retriever_prompt = ZeroShotAgent.create_prompt(
    tools=[retriever_tool], )

memory_retriever = ConversationBufferMemory(
        memory_key='history',
        return_messages=True,
)

#Set up the LLM
llm_chain_retriever = LLMChain(llm=chat_llm, prompt=retriever_prompt)
agent_retriever = ZeroShotAgent(llm_chain=llm_chain_retriever, tools=[retriever_tool], verbose=True)

#Integrating tools, agent, and memory
agent_chain_retriever = AgentExecutor.from_agent_and_tools(agent=agent_retriever, tools=[retriever_tool], verbose=True, memory=memory_retriever, return_only_outputs=True)


# dummy prompt given that agent and multipromt routing doesnt work or Conversational retrieval
retriever_prompt2 = PromptTemplate(
    input_variables=['input'],
    template="return `{input}` do not add anything"
)

chain_retriever2 = LLMChain(llm=chat_llm, prompt=retriever_prompt2, output_key="text")

seq_chain_retriever = SimpleSequentialChain(chains=[agent_chain_retriever, chain_retriever2],output_key="text")

#print(chain_1.run({"input":"i want to buy a house"}))

/Users/matty/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.mrkl.base.ZeroShotAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  warn_deprecated(


## Apartment Purchase input Chain 

In [17]:
memory_purc = ConversationBufferMemory(memory_key="history",return_messages=True)


instructions= """your are a realtor chat agent who sells apartments, your duty is to respond to purchase request for any of the apartment or house. 
    
            `if the user wants to make a purchase ask them for their: name, id_number and the apartment number`.
            
            `make sure that these values are passed if not keep persisting for them`.
            
            `if all the values are inputed, 
            
            << FORMATTING >>
                Return a JSON object formatted to look like:
                ```
                {{
                    "name": string \ users name
                    "id_number": string \ users id_number
                    "apartment_id": string \ apartment id being purchased 
                }}```


            """

prompt_purc = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(instructions),
        # The `variable_name` here is what must align with memory
         MessagesPlaceholder(variable_name="history"),
        HumanMessagePromptTemplate.from_template("{input}"),
    ]
)

chain_purc= LLMChain(llm=chat_llm, prompt=prompt_purc,memory=memory_purc,output_key="text")


## Multiple Chain Pipeline Routing

In [18]:
prompt_infos = [
    {
        "name": "apartment info",
        "description": "You give information about available apartments/houses",
        "chain":  seq_chain_retriever,
    },
    {
        "name": "purchase apartment",
        "description": instructions,
        "chain": chain_purc,
    },

]

In [19]:
# map destination chains
destination_chains = {}
for prompt_info in prompt_infos:
    name = prompt_info["name"]
    destination_chains[name] = prompt_info['chain']
default_chain = ConversationChain(llm=chat_llm,output_key='text')


In [20]:
# Creating LLMRouterChain
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
# print(destinations_str)

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

In [21]:
# creating the router chain
router_chain = LLMRouterChain.from_llm(chat_llm, router_prompt)

# Multiple Prompt Chain
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True, 
)

In [22]:
print(chain.run({"input":"i want to buy an apartment"}))

/Users/matty/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new MultiPromptChain chain...
purchase apartment: {'input': 'i want to buy an apartment'}
> Finished chain.
That's great! Could you please provide me with your name, ID number, and the apartment number you are interested in?


In [23]:
class PurchaseReservationInput(BaseModel):
    """Inputs for making a purchase reservation"""
    apartment_id: str = Field(description="ID of the apartment")
    id_number: str = Field(description=" documents with the user id number")

class PurchaseReservationTool(BaseTool):
    name = "make_purchase_reservation"
    description = """
        Tool for making a purchase for an apartment.
        Prints out the provided data and SUCCESS.
        """
    args_schema: Type[BaseModel] = PurchaseReservationInput

    def _run(self, apartment_id: str, id_number: str):
        print("Apartment ID:", apartment_id)
        print("ID Number:", id_number)
        print("SUCCESS")



tool_purc = [
     
   PurchaseReservationTool()
]

agent_make_purc = initialize_agent(
    tool_purc, chat_llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

agent_make_purc.run("""Thank you for providing all the necessary information, James. Here is the purchase request:

```
{
    "name": "james",
    "id_number": "12345",
    "apartment_id": "5"
}```""")


/Users/matty/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `make_purchase_reservation` with `{'apartment_id': '5', 'id_number': '12345'}`


Apartment ID: 5
ID Number: 12345
SUCCESS
NoneThe purchase reservation for the apartment has been successfully made, James.

> Finished chain.


'The purchase reservation for the apartment has been successfully made, James.'

In [24]:
def chatRelator(message:str):
    
    """
    This function is used to have a conversation with the realestate agent to get info on apartments and make a purchase
    """
    response = chain.run({"input":message})
    print(response)
    
    if '"name":' in response and '"id_number":' in response and '"apartment_id":' in response:
        resp = agent_make_purc.run(response)
        print(resp)
    else:
        pass
    

In [25]:
chatRelator("which apartment is numbers 101")



> Entering new MultiPromptChain chain...
apartment info: {'input': 'which apartment is number 101'}

> Entering new AgentExecutor chain...
I need to use the apartment info tool to find out which apartment is number 101.
Action: apartment info
Action Input: 101
Observation: apartment_id: 1
number: 101
address: 123 Main Street

apartment_id: 2
number: 102
address: 456 Elm Street

apartment_id: 10
number: 110
address: 802 Ash Street

apartment_id: 7
number: 107
address: 246 Birch Drive
Thought:I now know the final answer
Final Answer: The apartment number 101 is located at 123 Main Street.

> Finished chain.

> Finished chain.
The apartment number 101 is located at 123 Main Street.


### Conclusion

The **Custom Functional Agents** bot delivered the core function a and b. several approach were carried out to meet the projects goal. the bot is able to answer questions on the csv dummy dataset on housing/apartment. it can provide information about the apartments dataset. if users are requesting to make a purchase of an apartment, it collect some basic informations and executes a purchase function. Tho no validation was carried out for the scope of this project for making sure apartments exist or can be purchased. further improvement can be carried out in a more relaxable timeframe. due to the timeframe, core function c was skipped.

### Limitation

- Due to time constraints, several functionalities that could have been implemented in the system were not prioritized.
- The system for this test development can engage in basic conversation with a user regarding information about an apartment and making a purchase.
- The system does not validate whether the given input corresponds to an existing apartment or not.
- The prompt could be enhanced for better user interaction.
- The model may occasionally deviate from the intended path if the questions are unrelated.

### Challenges faced

- Integration Complexity: One major challenge encountered involved integrating multiple chains and agents, as each had outputs that were incompatible with one another.
- Dictionary Input Handling: Another challenge arose in effectively managing a dictionary input within a custom function.
- Sequential Agent Utilization: There was difficulty in efficiently utilizing agents and retrieval chains in a sequential manner, posing a significant challenge in the development process.